In [1]:
import pandas as pd
import os

In [2]:
cvs_fp = r'C:\Users\iya\Dev\CVM\CVS\2022'
cvs_tour_fn = r'sandag_cvs_tour.xlsx'
tnc_tour_fn = r'sandag_tnc_tour.xlsx'
cvs_df_fn = r'cv_20221102_sandag_draftfinal.xlsx'
db_fn = r'SANDAG 2022 CV DataBase & Dictionaires_03_03_2023.xlsx'

pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
establishment_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Establishment Data')
trip_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Trip Data')
tour_data = pd.read_excel(os.path.join(cvs_fp, cvs_tour_fn), sheet_name='sandag_cvs_tour')
veh_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Vehicle Data')



In [4]:
trip_data = trip_data.merge(
    veh_data[['vehicle_classification', 'id']],
    left_on='vehicle_id',
    right_on='id',
    validate='many_to_one',
    how='left'
    )
trip_data = trip_data[trip_data.vehicle_classification != 4]
trip_data


,company_id,vehicle_id,driver_id,trip_number,trip_load_status,trip_load_weight,activity_type,activity_type_other,placetype,placetype_other,...,arrival_time,departure_time,used_other_vehicle,participation_type,vehicle_id.1,Lower Estimate Weight Factor,Most Likely Estimate Weight Factor,Upper Estimate Weight Factor,vehicle_classification,id
0,100012,2062,2589,0,4.00,NaN,1,NaN,11,NaN,...,NaN,10:00:00,2,online,2062,109.69,177.83,198.18,1,2062
1,100012,2062,2589,1,NaN,NaN,11,NaN,6,NaN,...,10:20:00,11:10:00,2,online,2062,109.69,177.83,198.18,1,2062
2,100012,2062,2589,2,NaN,NaN,1,NaN,11,NaN,...,11:22:00,11:40:00,2,online,2062,109.69,177.83,198.18,1,2062
3,100012,2062,2589,3,NaN,NaN,10,NaN,96,DOG PARK,...,12:00:00,14:00:00,2,online,2062,109.69,177.83,198.18,1,2062
4,100012,2062,2589,4,NaN,NaN,1,NaN,11,NaN,...,14:20:00,NaN,2,online,2062,109.69,177.83,198.18,1,2062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12256,600154,1859,2398,2,NaN,NaN,6,NaN,20,NaN,...,08:46:00,09:11:00,2,smartphone,1859,178.85,247.70,254.34,8,1859
12257,600154,1859,2398,3,NaN,NaN,5,NaN,20,NaN,...,09:44:00,10:05:00,2,smartphone,1859,178.85,247.70,254.34,8,1859
12258,600154,1859,2398,4,NaN,NaN,10,NaN,96,ROAD SIDE,...,10:38:00,10:54:00,2,smartphone,1859,178.85,247.70,254.34,8,1859
12259,600154,1859,2398,5,NaN,NaN,5,NaN,20,NaN,...,11:27:00,11:46:00,2,smartphone,1859,178.85,247.70,254.34,8,1859


In [5]:
tour_data = tour_data[tour_data.VehClass != 4]

In [6]:
employees = establishment_data[['base_location_Industry Group', 'no_of_emp_work']]
employees = employees.groupby('base_location_Industry Group').sum()
employees

,no_of_emp_work
base_location_Industry Group,
1,1114
2,6618
3,1025
4,6040
5,2852
6,3785
7,2133
8,6264
9,8220


In [7]:
trip = trip_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')
gc = trip[
    ['base_location_Industry Group', 'Most Likely Estimate Weight Factor']
    ].groupby('base_location_Industry Group').sum()
gc

,Most Likely Estimate Weight Factor
base_location_Industry Group,
1,"12,155.78"
2,"120,978.74"
3,"21,123.99"
4,"94,282.14"
5,"79,692.94"
6,"200,942.20"
7,"96,814.66"
8,"259,008.39"
9,"538,669.76"


In [8]:
tour_data.columns

Index(['company_id', 'driver_id', 'trip_number', 'vehicle_id',
       'STOP_BASE_NOTBASE', 'TOUR_NUM', 'STOP_SEQ', 'trip_load_status',
       'trip_load_weight', 'activity_type', 'activity_type_other', 'placetype',
       'placetype_other', 'location_placename', 'location_address',
       'location_city', 'location_state', 'location_zip', 'cargo_pickup',
       'cargo_other_pickup', 'cargo_delivery', 'cargo_other_delivery',
       'cargo_pu_weight', 'cargo_do_weight', 'travel_date', 'arrival_time',
       'departure_time', 'used_other_vehicle', 'participation_type',
       'Lower.Estimate.Weight.Factor', 'Most.Likely.Estimate.Weight.Factor',
       'Upper.Estimate.Weight.Factor', 'TRAVEL_DISTANCE', 'TRAVEL_MINUTES',
       'stop_num', 'VehClass', 'StopPurpose', 'ActivityName', 'SORT_ORIG',
       'sort_clo', 'StopDuration', 'VehClassLMH', 'IndName', 'DistBin',
       'TimeBin', 'DistRound', 'TimeRound', 'OBJECTID', 'TAZ', 'Shape_Leng',
       'Shape_Area', 'geometry', 'TOUR_STOP_TAZID'

In [9]:

tour = tour_data[['company_id', 'TOUR_NUM']].groupby('company_id').nunique().reset_index()
tour = tour.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')
tour
#tour[['TOUR_NUM', 'base_location_Industry Group']].groupby('base_location_Industry Group').sum()

,company_id,TOUR_NUM,base_location_Industry Group
0,100012,2,9
1,100021,1,1
2,100023,1,4
3,100039,1,1
4,100050,1,1
...,...,...,...
449,600041,2,8
450,600051,2,4
451,600056,1,6
452,600145,3,10


In [10]:
tour_data[['company_id','Most.Likely.Estimate.Weight.Factor']].groupby('company_id').nunique()

,Most.Likely.Estimate.Weight.Factor
company_id,
100012,1
100021,1
100023,1
100039,1
100050,1
...,...
600041,2
600051,1
600056,1


In [11]:
tour_id = []
counter = 0
start, end = tour_data['TOUR_START'], tour_data['TOUR_END']

for se in list(zip(start, end)):
    if se == (True, False):
        counter += 1
    tour_id.append(counter)

In [12]:
tour_data['tour_id'] = tour_id

In [13]:
tour_data = tour_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')

In [14]:
tour_1 = tour_data[['Most.Likely.Estimate.Weight.Factor', 'tour_id']].groupby('tour_id').mean().reset_index()
tour_1 = tour_1.merge(
    tour_data[['base_location_Industry Group', 'tour_id']].groupby('tour_id').mean(), 
    on='tour_id',
    how='left',
    validate='one_to_many'
    )
tour_1[
    ['Most.Likely.Estimate.Weight.Factor', 'base_location_Industry Group']
].groupby('base_location_Industry Group').sum().reset_index()

,base_location_Industry Group,Most.Likely.Estimate.Weight.Factor
0,1.00,"2,386.82"
1,2.00,"22,801.71"
2,3.00,"5,409.18"
3,4.00,"18,768.58"
4,5.00,"13,948.57"
5,6.00,"42,326.07"
6,7.00,"11,813.85"
7,8.00,"55,873.74"
8,9.00,"84,795.47"
9,10.00,"47,738.45"


In [15]:
vmt = tour_data[
    ['Most.Likely.Estimate.Weight.Factor', 'base_location_Industry Group', 'TRAVEL_DISTANCE']
].dropna(how='any')
vmt['weighted_distance'] = vmt['Most.Likely.Estimate.Weight.Factor'] * vmt.TRAVEL_DISTANCE
vmt[['weighted_distance', 'base_location_Industry Group']].groupby('base_location_Industry Group').sum().reset_index()

,base_location_Industry Group,weighted_distance
0,1,"94,294.62"
1,2,"1,455,627.51"
2,3,"94,476.61"
3,4,"662,266.36"
4,5,"1,054,351.58"
5,6,"1,996,970.71"
6,7,"700,589.36"
7,8,"1,955,834.77"
8,9,"2,830,538.79"
9,10,"3,162,765.30"


In [16]:
light, med, hvy = list(range(1, 4)), list(range(5, 8)), [4, 8]
def classing(num):
    if num in light: return 'light'
    elif num in med: return 'med'
    else: return 'hvy'
tour_data['VehClass_name'] = tour_data.VehClass.apply(lambda x: classing(x))

vmt = tour_data[
    ['Most.Likely.Estimate.Weight.Factor', 'base_location_Industry Group', 'TRAVEL_DISTANCE', 'VehClass_name']
].dropna(how='any')
vmt = vmt[vmt.VehClass_name != 'light']
vmt['weighted_distance'] = vmt['Most.Likely.Estimate.Weight.Factor'] * vmt.TRAVEL_DISTANCE
vmt[['weighted_distance', 'base_location_Industry Group']].groupby('base_location_Industry Group').sum().reset_index()


,base_location_Industry Group,weighted_distance
0,1,"75,846.32"
1,2,"1,252,495.63"
2,3,"53,948.85"
3,4,"381,875.91"
4,5,"412,153.19"
5,6,"1,007,088.57"
6,7,"314,949.34"
7,8,"430,478.58"
8,9,"1,205,466.61"
9,10,"1,283,703.38"
